In [1]:
# Install necessary libraries
!pip install flask-ngrok flask transformers

import os
import random
import json
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)


Loading Pre-trained Model
We will use a pre-trained transformer model for the chatbot.

In [2]:
# Load pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Chat history to maintain context
chat_history = {}

def generate_response(user_id, user_input):
    if user_id not in chat_history:
        chat_history[user_id] = []

    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([torch.tensor(chat_history[user_id]), new_user_input_ids], dim=-1) if chat_history[user_id] else new_user_input_ids

    chat_history[user_id] = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(chat_history[user_id][:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Implementing Proactive Assistance
The chatbot will send reminders or suggest services based on user activity.

In [3]:
import threading
import time

user_activity = {}

def track_user_activity(user_id, activity):
    if user_id not in user_activity:
        user_activity[user_id] = []

    user_activity[user_id].append(activity)

def proactive_assistance():
    while True:
        current_time = time.time()
        for user_id, activities in user_activity.items():
            for activity in activities:
                # Check for upcoming deadlines (dummy logic for demonstration)
                if current_time - activity['time'] > 60:  # 1 minute
                    response = f"Reminder: You have an upcoming deadline for {activity['task']}."
                    print(f"Proactive message to {user_id}: {response}")
                    # Send response to user (implementation depends on frontend integration)
                    activities.remove(activity)
        time.sleep(10)

# Start proactive assistance thread
threading.Thread(target=proactive_assistance, daemon=True).start()


Implementing Multi-Language Support
We will use a simple dictionary-based translation for demonstration. In a real-world application, you might use an API or a more sophisticated localization library.

In [4]:
# Simple translation dictionary (example)
translations = {
    'en': {
        'hello': 'Hello!',
        'how_can_i_help': 'How can I help you?'
    },
    'es': {
        'hello': '¡Hola!',
        'how_can_i_help': '¿Cómo puedo ayudarte?'
    }
}

def get_translation(user_id, key):
    # Assume user_locale is stored somewhere (dummy logic)
    user_locale = 'es' if random.random() > 0.5 else 'en'
    return translations.get(user_locale, {}).get(key, key)

# Flask routes
@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_id = data['user_id']
    user_input = data['message']

    response = generate_response(user_id, user_input)
    track_user_activity(user_id, {'task': user_input, 'time': time.time()})

    return jsonify({'response': response})

@app.route('/localized_response', methods=['GET'])
def localized_response():
    user_id = request.args.get('user_id')
    key = request.args.get('key')

    response = get_translation(user_id, key)

    return jsonify({'response': response})

# Run Flask app
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll